In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
spark = SparkSession.builder.appName("CDC pipeline").getOrCreate()

In [ ]:
fldf=spark.read.csv("/content/LOAD00000001.csv")
fldf=fldf.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","City")
fldf.write.mode("overwrite").csv("/content/final_output/finalfile.csv")

In [ ]:
udf=spark.read.csv("/content/20250717-080727667.csv")
udf=udf.withColumnRenamed("_c0","action").withColumnRenamed("_c1","id").withColumnRenamed("_c2","FullName").withColumnRenamed("_c3","City")
ffdf=spark.read.csv("/content/final_output/finalfile.csv")
ffdf=ffdf.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","City")

In [ ]:
for row in udf.collect():

  if row.action=="I":
    rowData=[list(row)[1:]]
    columns=["id","FullName","City"]
    tempDf=spark.createDataFrame([(row.id,row.FullName,row.City)],columns)
    ffdf=ffdf.union(tempDf)

  elif row.action=="U":
    ffdf=ffdf.withColumn("FullName",when(col("id")==row.id,row.FullName).otherwise(col("FullName"))).withColumn("City",when(col("id")==row.id,row.City).otherwise(col("City")))

  elif row.action=="D":
    ffdf=ffdf.filter(col("id")!=row.id)

# fldf.write.mode("overwrite").csv("/content/final_output/finalfile.csv")